# Uniswap calculation

This notebook is available at http://ai.hkphysics.hk/ username: user

This uses a modified uniswappy which is available on branch main at https://github.com/hkphysics/uniswappy

See

https://blog.uniswap.org/uniswap-v3-math-primer

https://atiselsts.github.io/pdfs/uniswap-v3-liquidity-math.pdf

https://medium.com/@icmoore/simulating-a-liquidity-pool-for-decentralized-finance-6f357ec8564b

In [ ]:
# set up calculations

import traceback
from dataclasses import dataclass

import numpy as np
import pandas as pd
from axc.algobot import BotSimulator
from axc.liquidity import LiquidityBot
from axc.lp import *
from icecream import ic

from uniswappy import *
from axc.algobot import BotSimulator
from axc.liquidity import LiquidityBot

tenv = token_scenario_baseline

In [ ]:
def setup_pool(tenv, pool):
    (lp, tkn0, tkn1) = setup_lp(tenv)
    adapter = BotSimulator(
        lp,
        tenv.user,
        tkn0,
        tkn1,
        [LiquidityBot(LiquidityBot.Params(pool_params=pool))],
    )
    adapter.init_step()
    return (lp, tkn0, tkn1)


def do_calc(
    tenv: TokenScenario, pool_params: list[float], pools, extra
) -> pd.DataFrame:
    """
    Run the swap for different pools
    """
    swap_points = np.geomspace(100, tenv.usdt_in, num=100)

    def process_pool(pool_param, swap):
        lp_result, tkn0, _ = setup_pool(tenv, pools(pool_param))
        try:
            out = Swap().apply(lp_result, tkn0, tenv.user, swap)
            return {
                "lower": pool_param,
                "swap": swap,
                "out": out,
                "price": float(out) / float(swap),
            } | extra(lp_result)
        except AssertionError as e:
            print(f"Skipping invalid value for {lower}: {e}")
            return None

    results = filter(
        None,
        [process_pool(pool_param, swap) for pool_param in pool_params for swap in swap_points],
    )
    return pd.DataFrame(results)

In [ ]:
results = do_calc(
    tenv,
    [0.95, 0.9, 0.8, 0.7, 0.5, 0.1, 0.000001],
    lambda lower: [[tenv.user_lp, "min_tick", "max_tick"], [tenv.reserve, lower, 1.0]],
    lambda lp: {},
)
plotme(
    results,
    "pool lower bound",
    """Scenario

Price %f
reserve %d TKN
one side pool from lower to 1.0
"""
    % (1.0, tenv.reserve),
)

In [ ]:
insurance_lower = 0.95
results = do_calc(
    tenv,
    np.geomspace(0.0001, 0.99, num=10),
    lambda lower: [
        [tenv.reserve * (1.0 - lower), "min_tick", "max_tick"],
        [tenv.reserve * lower, insurance_lower, 1.0],
    ],
    lambda lp: {},
)
plotme(
    results,
    "percent in insurance pool",
    """Scenario

Price %f
reserve %d TKN
assume percent in insurance and rest in v2 style pool
"""
    % (1.0, tenv.reserve),
)

In [ ]:
import matplotlib.pyplot as plt


def plotme2(df, title, annotation):
    plt.figure(figsize=(10, 6))
    for key, grp in df.groupby("lower"):
        plt.plot(grp["swap"], grp["y"], label=key)

    plt.text(25000, 0.6, annotation)
    plt.legend(title=title)
    plt.xlabel("Tokens sold (TKN)")
    plt.ylabel("Swap price (USDT/TKN)")
    plt.show()


def calc_tokens(lp):
    solver = SolveDeltas(lp)
    (x, y) = solver.calc(0.9998)
    return {"x": x, "y": y}


results = do_calc(
    tenv,
    [0.95, 0.9, 0.8, 0.7, 0.5, 0.1, 0.000001],
    lambda lower: [
        [tenv.user_lp, "min_tick", "max_tick"],
        [tenv.reserve, lower, tenv.nav],
    ],
    calc_tokens,
)

results
plotme2(results, "", "")

In [ ]:
(lp, tkn0, tkn1) = setup_pool(
    tenv,
    [
        [tenv.user_lp, "min_tick", "max_tick"],
        [tenv.reserve, 0.98, 1.0],
    ],
)
lp.get_positions_for_owner(tenv.user)